<a href="https://colab.research.google.com/github/anegrill/IDSN544/blob/main/IDSN_544_fast_ai_C4%20-%20alt%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'd/aaronnegrillo/yelp-review-full-xiang-zhang-et-al-2015:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5736311%2F9439878%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240921%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240921T043031Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4214ffcf0f332da5694cc7b95325d946f11b93e275352463ee4d91812f7997d9f358182f7296d19d50d4fc871028a140f78d8831eb09986a70060d488af20d36673fb7e9c85fc6ee0a2cdeb5bf0db15380f390606859ab6bb5cbc99e6deec9b35ffe65e5b5f4fc740318fba0242038ef7a36c4cbae8a83dbe9475872702853a2b4675fdce0ac5fa163830036c9521c1c15118071a22353840abd9b6368a563ff48510e2521899c7fe7290c66a7dc6631f10982744e8eb2dcd97358209d91bdd1df6814bbe608504b3c119be2f052850983ccdaf84afacf3a65ae0bed8df7379206413b9e8d0ec042d7bd844d3185587e8faaf2787962a2133e997735da7d0dd0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 199990031 bytes downloaded
Downloaded and uncompressed: d/aaronnegrillo/yelp-review-full-xiang-zhang-et-al-2015
Data source import complete.


Aaron Negrillo <br>
IDSN 544, Fall 2024 <br>
anegrill@USC.edu

### fast.ai Chapter 4 Exercise - Natural Language Processing

For this exercise, we will use the fast.ai library to build a sample NLP model with the `AMAZON_REVIEWS` dataset.

In [2]:
# pip install fastai --upgrade

In [2]:
# Import necessary modules
from fastai.text.all import *
from fastai.tabular.all import *
from fastai.data.core import *
import os
import torch
from torch.utils.data import *
from sklearn.model_selection import train_test_split

In [3]:
# Define the path to the dataset
data_path = '../input/d/aaronnegrillo/yelp-review-full-xiang-zhang-et-al-2015/'

In [4]:
# List files in the dataset directory
files = os.listdir(data_path)
print(files)

['readme.txt', 'train.csv', 'test.csv']


In [5]:
# Load train and test datasets
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))

In [6]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649999 entries, 0 to 649998
Data columns (total 2 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Non-Null Count   Dtype 
---  ------                                                                                                                                                                                                                                                                                                                          

In [7]:
# Preview the Dataframe
train_df.head(5)

,5,"dr. goldberg offers everything i look for in a general practitioner. he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first. really, what more do you need? i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."
0,2,"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff. It seems that his staff simply never answers the phone. It usually takes 2 hours of repeated calling to get an answer. Who has time for that or wants to deal with it? I have run into this problem with many other doctors and I just don't get it. You have office workers, you have patients with medical needs, why isn't anyone answering the phone? It's incomprehensible and not work the aggravation. It's with regret that I..."
1,4,"Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."
2,4,Got a letter in the mail last week that said Dr. Goldberg is moving to Arizona to take a new position there in June. He will be missed very much. \n\nI think finding a new doctor in NYC that you actually like might almost be as awful as trying to find a date!
3,1,"I don't know what Dr. Goldberg was like before moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you c..."
4,5,Top notch doctor in a top notch practice. Can't say I am surprised when I was referred to him by another doctor who I think is wonderful and because he went to one of the best medical schools in the country. \nIt is really easy to get an appointment. There is minimal wait to be seen and his bedside manner is great.


We seem to be missing headers, so let's assign those

In [8]:
# Set column names
column_names = ['Score','Review']

# Re-define our Dataframe and assign headers
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'), header=None, names=column_names)

In [9]:
# Preview the *updated* Dataframe
train_df.head(5)

,Score,Review
0,5,"dr. goldberg offers everything i look for in a general practitioner. he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first. really, what more do you need? i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."
1,2,"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff. It seems that his staff simply never answers the phone. It usually takes 2 hours of repeated calling to get an answer. Who has time for that or wants to deal with it? I have run into this problem with many other doctors and I just don't get it. You have office workers, you have patients with medical needs, why isn't anyone answering the phone? It's incomprehensible and not work the aggravation. It's with regret that I..."
2,4,"Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."
3,4,Got a letter in the mail last week that said Dr. Goldberg is moving to Arizona to take a new position there in June. He will be missed very much. \n\nI think finding a new doctor in NYC that you actually like might almost be as awful as trying to find a date!
4,1,"I don't know what Dr. Goldberg was like before moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you c..."


In [10]:
# Pre-tokenize the text using the Tokenizer
tokenizer = Tokenizer.from_df(text_cols='Review')
tokens = tokenizer(train_df['Review'])

In [11]:
# Create Vocabulary Manually
# Flatten the list of tokens and get unique tokens
all_tokens = [token for sublist in tokens for token in sublist]
unique_tokens = set(all_tokens)

# Create a mapping of token to index
vocab = {token: idx + 1 for idx, token in enumerate(unique_tokens)}  # Start indices at 1

In [12]:
# Numericalize the Tokens
def numericalize(token_list):
    return [vocab.get(token, 0) for token in token_list]  # Use 0 for unknown tokens

numericalized_data = [numericalize(toks) for toks in tokens]

In [13]:
# Add numericalized reviews to the dataframe
train_dfN = train_df
train_dfN['Review'] = numericalized_data

In [14]:
train_dfN.head(1)

,Score,Review
0,5,"[93, 13, 42, 65, 15, 83, 37, 93, 12, 74, 13, 15, 65, 83, 73, 73, 74, 13, 51, 65, 74, 31, 74, 13, 5, 85, 11, 6, 29, 15, 65, 6, 65, 37, 83, 83, 88, 65, 73, 83, 13, 65, 6, 29, 65, 78, 65, 15, 74, 29, 74, 13, 78, 37, 65, 19, 13, 78, 27, 85, 6, 85, 6, 83, 29, 74, 13, 42, 65, 65, 11, 74, 14, 51, 65, 29, 6, 27, 74, 65, 78, 29, 93, 65, 74, 78, 51, 5, 65, 85, 83, 65, 85, 78, 37, 88, 65, 85, 83, 65, ...]"


In [ ]:
dls_lm = TextDataLoaders.from_df(
    train_dfN,               # The dataframe
    text_col='Review',      # Column containing text data
    label_col='Score',      # Column containing labels
    is_lm=False,            # Set to False since it's a classification task
    valid_pct=0.2,          # Use 20% of the data for validation
    seed=42,                # For reproducibility
    seq_len=960,             # Sequence length of text
    bs=128                   # Batch size
)

In [ ]:
# Preview a batch to check everything is correct
dls.show_batch()

In [27]:
# # Define a custom Dataset class
# class TextDataset(Dataset):
#     def __init__(self, df):
#         self.df = df

#     def __len__(self):%qtconsole

#     def __getitem__(self, idx):
#         # Extract the numericalized review (text input) and the score (label)
#         review = self.df.iloc[idx]['Numericalized_Review']
#         score = self.df.iloc[idx]['Score']
#         # Convert to fastai's TensorText and TensorCategory
#         return TensorText(review), TensorCategory(score)

In [45]:
# Split the DataFrame into training and validation sets
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [40]:
# Create DataLoaders object for training a language model
dls_lm = DataLoaders(train_dl, train_dl)  # Use the same DataLoader for both train and valid for simplicity

In [46]:
# Create TextDataset instances for train and valid sets
train_ds = TextDataset(train_df)
valid_ds = TextDataset(valid_df)

In [48]:
# Create PyTorch DataLoader objects
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=64)

In [49]:
# Wrap the PyTorch DataLoaders into fastai DataLoaders
dls_lm = DataLoaders(train_dl, valid_dl)

In [51]:
dls_lm

In [52]:
# Preview the content of DataLoaders with show_batch
dls_lm.show_batch(max_n=5, truncate=100)

AttributeError: 'DataLoader' object has no attribute 'show_batch'

In [53]:
# Building the Learner
learnNLP = text_classifier_learner(
    dls_lm,
    AWD_LSTM,
    drop_mult=0.3,
    metrics=[accuracy, Perplexity()]
).to_fp16()

AttributeError: 'DataLoader' object has no attribute 'vocab'

In [ ]:
# # Intermediate Model Results
# learnNLP.fit_one_cycle(1, 2e-2

In [ ]:
# learnNLP.save('1epoch')
# learnNLP = learnNLP.load('1epoch')

In [ ]:
# learnNLP.unfreeze()
# learnNLP.fit_one_cycle(10, 2e-3)

In [ ]:
# learnNLP.save_encoder('finetuned')

In [ ]:
# dls.vocab

In [ ]:
# # Use the vocab from dls directly
# vocab = dls.vocab

# # Print the type and some sample values from the vocab
# print(f"Type of vocab: {type(vocab)}")
# print(f"Sample vocab items: {vocab[:10]}")  # Print first 10 items in the vocab

In [ ]:
# # Display the first few rows of the 'Review' column
# print(train_df['Review'].head())

In [ ]:
# # Define the DataBlock for classification using pretrained vocab
# dls_clas = DataBlock(
#     # blocks=(TextBlock.from_df('Review', vocab=vocab, seq_len=500), CategoryBlock),
#     blocks=(TextBlock.from_df('Review', seq_len=500), CategoryBlock),  # Text and target type
#     get_x=ColReader('Review'),  # Get the text from the 'Review' column
#     get_y=ColReader('Score'),   # Get the labels from the 'Score' column
#     splitter=RandomSplitter(valid_pct=0.2, seed=42)  # Define a random train/valid split
# )

In [ ]:
# # Build the DataLoaders
# dls_clas = dls_clas.dataloaders(train_df, bs=128)